# An Introduction to LlamaIndex Query Pipelines

## Overview
LlamaIndex provides a declarative query API that allows you to chain together different modules in order to orchestrate simple-to-advanced workflows over your data.

This is centered around our `QueryPipeline` abstraction. Load in a variety of modules (from LLMs to prompts to retrievers to other pipelines), connect them all together into a sequential chain or DAG, and run it end2end.

**NOTE**: You can orchestrate all these workflows without the declarative pipeline abstraction (by using the modules imperatively and writing your own functions). So what are the advantages of `QueryPipeline`? 

- Express common workflows with fewer lines of code/boilerplate
- Greater readability
- Greater parity / better integration points with common low-code / no-code solutions (e.g. LangFlow)
- [In the future] A declarative interface allows easy serializability of pipeline components, providing portability of pipelines/easier deployment to different systems.

## Cookbook

In this cookbook we give you an introduction to our `QueryPipeline` interface and show you some basic workflows you can tackle.

- Chain together prompt and LLM
- Chain together query rewriting (prompt + LLM) with retrieval
- Chain together a full RAG query pipeline (query rewriting, retrieval, reranking, response synthesis)

## Setup

Here we setup some data + indexes (from PG's essay) that we'll be using in the rest of the cookbook.

In [ ]:
# setup Arize Phoenix for logging/observability
import phoenix as px
px.launch_app()
import llama_index
llama_index.set_global_handler("arize_phoenix")

In [ ]:
from llama_index.query_pipeline.query import QueryPipeline
from llama_index.llms import OpenAI
from llama_index.prompts import PromptTemplate
from llama_index import (
    VectorStoreIndex, 
    ServiceContext,
    SimpleDirectoryReader,
    load_index_from_storage
)

In [ ]:
reader = SimpleDirectoryReader("../data/paul_graham")

In [ ]:
docs = reader.load_data()

In [ ]:
import os
from llama_index.storage import StorageContext

if not os.path.exists("storage"):
    index = VectorStoreIndex.from_documents(docs)
    # save index to disk
    index.set_index_id("vector_index")
    index.storage_context.persist("./storage")
else:
    # rebuild storage context
    storage_context = StorageContext.from_defaults(persist_dir="storage")
    # load index
    index = load_index_from_storage(storage_context, index_id="vector_index")

## 1. Chain Together Prompt and LLM 

In this section we show a super simple workflow of chaining together a prompt with LLM.

We simply define `chain` on initialization. This is a special case of a query pipeline where the components are purely sequential, and we automatically convert outputs into the right format for the next inputs.

In [ ]:
# try chaining basic prompts
prompt_str = "Please generate related movies to {movie_name}"
prompt_tmpl = PromptTemplate(prompt_str)
llm = OpenAI(model='gpt-3.5-turbo')

p = QueryPipeline(chain=[prompt_tmpl, llm], verbose=True)

In [ ]:
output = p.run(movie_name="The Departed")

> Running module 754bbc40-f642-4522-b838-6ad07fc4bcfb with input: 
movie_name: The Departed

> Running module e76f2960-d545-4116-abc0-7baa2feba63c with input: 
messages: Please generate related movies to The Departed



In [ ]:
print(str(output))

assistant: 1. Infernal Affairs (2002) - The Departed is actually a remake of this Hong Kong crime thriller. It follows a similar storyline of undercover cops infiltrating the criminal underworld.

2. The Town (2010) - Directed by Ben Affleck, this crime drama revolves around a group of bank robbers in Boston. It explores themes of loyalty, betrayal, and the blurred lines between law enforcement and criminals.

3. American Gangster (2007) - Based on a true story, this crime film follows the rise and fall of a Harlem drug lord and the detective determined to bring him down. It delves into the corrupt world of organized crime and the efforts to dismantle it.

4. Training Day (2001) - Denzel Washington won an Academy Award for his portrayal of a corrupt narcotics detective in this intense crime thriller. It showcases the moral dilemmas faced by law enforcement officers and the consequences of their actions.

5. The Departed (2006) - Although it's the same movie, it's worth mentioning it as

## Chain Together Query Rewriting Workflow (prompts + LLM) with Retrieval

Here we try a slightly more complex workflow where we send the input through two prompts before initiating retrieval.

1. Generate question about given topic.
2. Hallucinate answer given question, for better retrieval.

Since each prompt only takes in one input, note that the `QueryPipeline` will automatically chain LLM outputs into the prompt and then into the LLM. 

You'll see how to define links more explicitly in the next section.

In [ ]:
from llama_index.postprocessor import CohereRerank

# generate question regarding topic
prompt_str1 = "Please generate a concise question about Paul Graham's life regarding the following topic {topic}"
prompt_tmpl1 = PromptTemplate(prompt_str1)
# use HyDE to hallucinate answer.
prompt_str2 = (
    "Please write a passage to answer the question\n"
    "Try to include as many key details as possible.\n"
    "\n"
    "\n"
    "{query_str}\n"
    "\n"
    "\n"
    'Passage:"""\n'
)
prompt_tmpl2 = PromptTemplate(prompt_str2)

llm = OpenAI(model='gpt-3.5-turbo')
retriever = index.as_retriever(similarity_top_k=5)
p = QueryPipeline(chain=[prompt_tmpl1, llm, prompt_tmpl2, llm, retriever], verbose=True)

In [ ]:
nodes = p.run(topic="college")
len(nodes)

> Running module dcc5c91e-7e3e-4bf8-93e5-fb79b7b3f753 with input: 
topic: college

> Running module d3657f37-4e8b-45e7-a31c-a18b47f379b0 with input: 
messages: Please generate a concise question about Paul Graham's life regarding the following topic college

> Running module 5a961f21-b76b-47ae-a517-7227f852a2b3 with input: 
query_str: assistant: How did Paul Graham's college experience shape his career and entrepreneurial mindset?

> Running module 453dabbc-8a39-44a0-bfed-3a8027159765 with input: 
messages: Please write a passage to answer the question
Try to include as many key details as possible.


assistant: How did Paul Graham's college experience shape his career and entrepreneurial mindset?


Pass

> Running module f543ff83-c185-4a71-9aa7-96bfec15e403 with input: 
input: assistant: Paul Graham's college experience played a pivotal role in shaping his career trajectory and entrepreneurial mindset. As a student at Cornell University, Graham immersed himself in the world



5

## Chain Together a Full RAG Pipeline

Here we chain together a full RAG pipeline consisting of query rewriting, retrieval, reranking, and response synthesis.

Here we can't use `chain` syntax because certain modules depend on multiple inputs (for instance, response synthesis expects both the retrieved nodes and the original question). Instead we'll construct a DAG explicitly, through `add_modules` and then `add_link`.

In [ ]:
from llama_index.postprocessor import CohereRerank
from llama_index.response_synthesizers import TreeSummarize
from llama_index import ServiceContext

# define modules
prompt_str = "Please generate a question about Paul Graham's life regarding the following topic {topic}"
prompt_tmpl = PromptTemplate(prompt_str)
llm = OpenAI(model='gpt-3.5-turbo')
retriever = index.as_retriever(similarity_top_k=3)
reranker = CohereRerank()
summarizer = TreeSummarize(service_context=ServiceContext.from_defaults(llm=llm))

In [ ]:
# define query pipeline
p = QueryPipeline(verbose=True)
p.add_modules({
	"llm": llm,
	"prompt_tmpl": prompt_tmpl,
	"retriever": retriever,
    "summarizer": summarizer,
    "reranker": reranker
})
p.add_link("prompt_tmpl", "llm")
p.add_link("llm", "retriever")
p.add_link("retriever", "reranker", dest_key="nodes")
p.add_link("llm", "reranker", dest_key="query_str")
p.add_link("reranker", "summarizer", dest_key="nodes")
p.add_link("llm", "summarizer", dest_key="query_str")

In [ ]:
response = p.run(topic="YC")

> Running module prompt_tmpl with input: 
topic: YC

> Running module llm with input: 
messages: Please generate a question about Paul Graham's life regarding the following topic YC

> Running module retriever with input: 
input: assistant: What role did Paul Graham play in the founding and development of Y Combinator (YC)?

> Running module reranker with input: 
query_str: assistant: What role did Paul Graham play in the founding and development of Y Combinator (YC)?
nodes: [NodeWithScore(node=TextNode(id_='543f958b-2c46-4c0f-b046-22e0a60ea950', embedding=None, metadata={'file_path': '../data/paul_graham/paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file

> Running module summarizer with input: 
query_str: assistant: What role did Paul Graham play in the founding and development of Y Combinator (YC)?
nodes: [NodeWithScore(node=TextNode(id_='6b43fef2-821a-49a2-a043-9d816b70560f', embedding=None, metadata={'file_path': '../data/paul_graham/paul_graham_essay.txt', 'file_

In [ ]:
print(str(response))

Paul Graham played a significant role in the founding and development of Y Combinator (YC). He was one of the co-founders of YC and was actively involved in its early stages. He helped establish the Summer Founders Program (SFP) and was responsible for selecting and funding the initial batch of startups. Graham also played a key role in shaping the funding model for YC, based on previous deals and agreements. As YC grew, Graham's involvement expanded, and he dedicated a significant amount of his time and attention to the organization. However, in later years, he decided to step back from his role and hand over the leadership to Sam Altman.
